In [2]:
!pip install transformers==4.39.3

  Using cached transformers-4.39.3-py3-none-any.whl.metadata (134 kB)
  Using cached tokenizers-0.15.2-cp312-none-win_amd64.whl.metadata (6.8 kB)
Using cached transformers-4.39.3-py3-none-any.whl (8.8 MB)
Using cached tokenizers-0.15.2-cp312-none-win_amd64.whl (2.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.0.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.39.3 which is incompatible.


In [4]:
!pip install sentence-transformers

  Using cached transformers-4.53.3-py3-none-any.whl.metadata (40 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached transformers-4.53.3-py3-none-any.whl (10.8 MB)
Using cached tokenizers-0.21.2-cp39-abi3-win_amd64.whl (2.5 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3


In [6]:
!pip install tqdm

In [8]:
!pip install tf-keras

In [10]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
tqdm.pandas()


In [14]:
# Load CSV file
file_path = "C://Users//Shaban Hassan//OneDrive//Desktop//internship//all_tickets_processed_improved_v3.csv"
df = pd.read_csv(file_path)

# Show a sample
df.head()


,Document,Topic_group
0,connection with icon icon dear please setup ic...,Hardware
1,work experience user work experience user hi w...,Access
2,requesting for meeting requesting meeting hi p...,Hardware
3,reset passwords for external accounts re expir...,Access
4,mail verification warning hi has got attached ...,Miscellaneous


In [18]:
# Extract all unique tags (topics)
labels = df['Topic_group'].dropna().unique().tolist()
print("Available Tags:\n", labels)


Available Tags:
 ['Hardware', 'Access', 'Miscellaneous', 'HR Support', 'Purchase', 'Administrative rights', 'Storage', 'Internal Project']


In [32]:
from transformers import pipeline

# Initialize zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors: 100%|#########9| 1.63G/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [34]:
def classify_ticket(text):
    result = classifier(text, candidate_labels=labels, multi_label=True)
    # Extract top 3 predicted labels
    top_3 = [result['labels'][i] for i in range(min(3, len(result['labels'])))]
    return top_3


In [74]:
# Sample tickets to test
df_sample = df.sample(500, random_state=42)

# Apply the classification function
df_sample['zero_shot_top_3'] = df_sample['Document'].apply(classify_ticket)

# Show result
df_sample[['Document', 'zero_shot_top_3']]


,Document,zero_shot_top_3
41946,issue with server process sent tuesday issue w...,"[Internal Project, Miscellaneous, Access]"
20882,new purchase po wednesday february purchase po...,"[Internal Project, Storage, Administrative rig..."
37017,oracle pas urgent re we updated rights query h...,"[Storage, Administrative rights, Purchase]"
29138,new starter access thursday pm re starter form...,"[Storage, Miscellaneous, Hardware]"
9536,new phone phone hi taken work mobile phone spe...,"[Internal Project, Administrative rights, HR S..."
...,...,...
9427,patch cable request patch cable hello patch co...,"[Purchase, Internal Project, Miscellaneous]"
13059,rebuild en friday pm rebuild assets hi guys pl...,"[Internal Project, HR Support, Storage]"
2874,add to add hi please add thank head,"[Internal Project, Administrative rights, HR S..."
34897,its report for the user report for hi please c...,"[Hardware, Storage, Purchase]"


In [76]:
# Define a function to check if the true label is in top 3 predictions
def is_correct(row):
    return row['Topic_group'] in row['zero_shot_top_3']

# Apply this function to each row
df_sample['correct_zero_shot'] = df_sample.apply(is_correct, axis=1)


In [78]:
# Calculate mean accuracy
accuracy = df_sample['correct_zero_shot'].mean()
print(f"Zero-Shot Top-3 Accuracy (Sample): {accuracy:.2f}")


Zero-Shot Top-3 Accuracy (Sample): 0.28


In [80]:
# View sample with correctness
df_sample[['Document', 'Topic_group', 'zero_shot_top_3', 'correct_zero_shot']]


,Document,Topic_group,zero_shot_top_3,correct_zero_shot
41946,issue with server process sent tuesday issue w...,Hardware,"[Internal Project, Miscellaneous, Access]",False
20882,new purchase po wednesday february purchase po...,Purchase,"[Internal Project, Storage, Administrative rig...",False
37017,oracle pas urgent re we updated rights query h...,Hardware,"[Storage, Administrative rights, Purchase]",False
29138,new starter access thursday pm re starter form...,HR Support,"[Storage, Miscellaneous, Hardware]",False
9536,new phone phone hi taken work mobile phone spe...,Hardware,"[Internal Project, Administrative rights, HR S...",False
...,...,...,...,...
9427,patch cable request patch cable hello patch co...,Hardware,"[Purchase, Internal Project, Miscellaneous]",False
13059,rebuild en friday pm rebuild assets hi guys pl...,Hardware,"[Internal Project, HR Support, Storage]",False
2874,add to add hi please add thank head,Miscellaneous,"[Internal Project, Administrative rights, HR S...",False
34897,its report for the user report for hi please c...,Access,"[Hardware, Storage, Purchase]",False
